In [19]:
import numpy as np
from pole_gen.pole_gen import generate_pole, normalize_mesh
import open3d as o3d
from utils.plot import plot_open3d
import random

pole_mesh = generate_pole()
pole_mesh.scale(random.uniform(1.0, 1.955), center=(0, 0, 0))
lamp_mesh = o3d.io.read_triangle_mesh("pole_gen/meshes/lamp.ply")
lamp_mesh.rotate(
    lamp_mesh.get_rotation_matrix_from_xyz(
        (0, np.deg2rad(random.uniform(-5.0, 0.0)), 0)
    ),
    center=(0, 0, 0),
)
lamp_mesh.translate(
    [0, 0, 7.5 + random.uniform(-1.0, 1.94)]
)  # TODO Make sure this doesnt go higher than the pole

mesh = pole_mesh + lamp_mesh
# mesh.scale(random.uniform(0.75, 1.06), center=(0, 0, 0))
normalize_mesh(mesh)
plot_open3d([mesh])